In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
num_classes = 30  
batch_size = 32

In [3]:
data_dir = 'Segmented Medicinal Leaf Images'

In [4]:
# Créez un générateur d'images avec augmentation de données
data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% des données pour la validation
)
input_shape = (224, 224, 3)


In [5]:
# Chargez les données d'entraînement et de validation en utilisant le générateur
train_generator = data_generator.flow_from_directory(
    data_dir,
    target_size = input_shape[:2],
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Utilisez la sous-partie d'entraînement
)

Found 1479 images belonging to 30 classes.


In [6]:
valid_generator = data_generator.flow_from_directory(
    data_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Utilisez la sous-partie de validation
)

Found 356 images belonging to 30 classes.


In [7]:
# Chargement du modèle pré-entraîné InceptionV3 avec les poids ImageNet
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

In [8]:
# Ajout de couches personnalisées au modèle InceptionV3
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)


In [9]:
# Création du modèle final
model = Model(inputs=base_model.input, outputs=predictions)


In [10]:
# Compilation du modèle
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Utilisation de l'arrêt précoce
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [13]:
# Entraînement du modèle
model.fit(
    train_generator,
    epochs=50,
    callbacks=[early_stopping],
    validation_data=valid_generator  # Remplacez par votre générateur de validation
)

Epoch 1/50
47/47 [==============================] - 888s 18s/step - loss: 2.2886 - accuracy: 0.4260 - val_loss: 1.2378 - val_accuracy: 0.6461
Epoch 2/50
47/47 [==============================] - 768s 16s/step - loss: 0.4984 - accuracy: 0.8952 - val_loss: 0.4931 - val_accuracy: 0.8371
Epoch 3/50
47/47 [==============================] - 720s 15s/step - loss: 0.1901 - accuracy: 0.9554 - val_loss: 0.2734 - val_accuracy: 0.9185
Epoch 4/50
47/47 [==============================] - 773s 16s/step - loss: 0.1027 - accuracy: 0.9750 - val_loss: 0.1198 - val_accuracy: 0.9691
Epoch 5/50
47/47 [==============================] - 694s 15s/step - loss: 0.0946 - accuracy: 0.9777 - val_loss: 0.1128 - val_accuracy: 0.9719
Epoch 6/50
47/47 [==============================] - 701s 15s/step - loss: 0.0645 - accuracy: 0.9851 - val_loss: 0.1565 - val_accuracy: 0.9663
Epoch 7/50
47/47 [==============================] - 733s 16s/step - loss: 0.0713 - accuracy: 0.9831 - val_loss: 0.1103 - val_accuracy: 0.9747
Epoch 

In [20]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model.evaluate(valid_generator)  
print(f"Test InceptionV3 accuracy: {test_accuracy * 100:.2f}%")

12/12 [==============================] - 39s 3s/step - loss: 0.1132 - accuracy: 0.9747
Test InceptionV3 accuracy: 97.47%
